In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from scipy.stats import linregress
import scipy.stats as stats

# Import API key
from api_keys import g_key

In [2]:
#Import location data set
df_city_list = pd.read_csv("../Resources/City_loc.csv")
df_city_list

,LAD,AreaCode,CTY,CTY_Code,Rgn,Rgn_Code,lat,long
0,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494
1,Allerdale,E07000026,Cumbria,E10000006,North West,E12000002,54.844865,-3.318219
2,Amber Valley,E07000032,Derbyshire,E10000007,East Midlands,E12000004,53.029693,-1.339976
3,Arun,E07000224,West Sussex,E10000032,South East,E12000008,50.840609,-0.687156
4,Ashfield,E07000170,Nottinghamshire,E10000024,East Midlands,E12000004,53.101539,-1.225497
...,...,...,...,...,...,...,...,...
357,Rhondda Cynon Taf,W06000016,0,0,0,0,51.719100,-3.432545
358,Swansea,W06000011,0,0,0,0,51.692853,-3.900801
359,Torfaen,W06000020,0,0,0,0,51.729256,-3.064972
360,Vale of Glamorgan,W06000014,0,0,0,0,51.468172,-3.635452


In [6]:
df = pd.read_csv("../Resources/UK-HPI-full-file-2021-09.csv")
df

,Date,RegionName,AreaCode,AveragePrice,Index,IndexSA,1m%Change,12m%Change,AveragePriceSA,SalesVolume,...,NewPrice,NewIndex,New1m%Change,New12m%Change,NewSalesVolume,OldPrice,OldIndex,Old1m%Change,Old12m%Change,OldSalesVolume
0,01/01/2004,Aberdeenshire,S12000034,81693.66964,40.864214,NaN,NaN,NaN,NaN,388.0,...,88436.13953,40.267250,NaN,NaN,103.0,81043.95084,40.883367,NaN,NaN,285.0
1,01/02/2004,Aberdeenshire,S12000034,81678.76231,40.856757,NaN,-0.018248,NaN,NaN,326.0,...,88606.44649,40.344795,0.192576,NaN,107.0,80965.29542,40.843688,-0.097053,NaN,219.0
2,01/03/2004,Aberdeenshire,S12000034,83525.09702,41.780317,NaN,2.260483,NaN,NaN,453.0,...,90296.91375,41.114508,1.907838,NaN,140.0,82903.23948,41.821302,2.393549,NaN,313.0
3,01/04/2004,Aberdeenshire,S12000034,84333.67900,42.184780,NaN,0.968071,NaN,NaN,571.0,...,90319.87844,41.124964,0.025432,NaN,180.0,84003.99161,42.376586,1.327755,NaN,391.0
4,01/05/2004,Aberdeenshire,S12000034,86379.95396,43.208353,NaN,2.426403,NaN,NaN,502.0,...,91989.17763,41.885039,1.848208,NaN,167.0,86222.73484,43.495852,2.641235,NaN,335.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133561,01/05/2021,Yorkshire and The Humber,E12000003,182339.40770,133.906985,132.158707,1.524226,10.528659,179958.7996,5411.0,...,237438.53910,142.986271,-0.369836,10.255076,59.0,179017.74980,133.360876,1.721641,10.699447,5352.0
133562,01/06/2021,Yorkshire and The Humber,E12000003,193209.60250,141.889873,139.804564,5.961517,15.046007,190370.0638,5465.0,...,247646.46220,149.133515,4.299185,19.493385,63.0,189999.57850,141.541887,6.134492,14.597427,5402.0
133563,01/07/2021,Yorkshire and The Humber,E12000003,181001.68730,132.924586,130.674281,-6.318483,7.266898,177937.4752,1811.0,...,248964.56800,149.927283,0.532253,16.371731,6.0,176703.34290,131.636737,-6.998034,6.435467,1805.0
133564,01/08/2021,Yorkshire and The Humber,E12000003,184981.58050,135.847353,133.147731,2.198816,8.176328,181305.5409,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Format the date data type
df.dtypes
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Date'].dtypes

dtype('<M8[ns]')

In [4]:
counties = pd.read_csv("../Resources/County_Codes.csv")
counties

,Rgn_Code,RegionName
0,E10000003,Cambridgeshire
1,E10000006,Cumbria
2,E10000007,Derbyshire
3,E10000008,Devon
4,E10000011,East Sussex
5,E10000012,Essex
6,E10000013,Gloucestershire
7,E11000001,Greater Manchester
8,E10000014,Hampshire
9,E10000015,Hertfordshire


In [35]:
counties.to_csv('../Clean_csv/counties.csv')

In [9]:
df_sales_volume = df.loc[:,["Date","AreaCode","RegionName","SalesVolume", "AveragePrice"]]
df_sales_volume

,Date,AreaCode,RegionName,SalesVolume,AveragePrice
0,2004-01-01,S12000034,Aberdeenshire,388.0,81693.66964
1,2004-02-01,S12000034,Aberdeenshire,326.0,81678.76231
2,2004-03-01,S12000034,Aberdeenshire,453.0,83525.09702
3,2004-04-01,S12000034,Aberdeenshire,571.0,84333.67900
4,2004-05-01,S12000034,Aberdeenshire,502.0,86379.95396
...,...,...,...,...,...
133561,2021-05-01,E12000003,Yorkshire and The Humber,5411.0,182339.40770
133562,2021-06-01,E12000003,Yorkshire and The Humber,5465.0,193209.60250
133563,2021-07-01,E12000003,Yorkshire and The Humber,1811.0,181001.68730
133564,2021-08-01,E12000003,Yorkshire and The Humber,NaN,184981.58050


In [29]:
#Format the data set to 2021-07-01 to show the latest value of sales volume and average price


housing_sales_volume_2021_09=df_sales_volume[(df_sales_volume['Date'] > "2020-01-01") & (df_sales_volume['Date'] < '2021-07-01')]
housing_sales_volume_2021_09

,Date,AreaCode,RegionName,SalesVolume,AveragePrice
193,2020-02-01,S12000034,Aberdeenshire,213.0,176096.5044
194,2020-03-01,S12000034,Aberdeenshire,275.0,174350.1581
195,2020-04-01,S12000034,Aberdeenshire,133.0,174855.7985
196,2020-05-01,S12000034,Aberdeenshire,150.0,176189.6206
197,2020-06-01,S12000034,Aberdeenshire,211.0,179137.7036
...,...,...,...,...,...
133558,2021-02-01,E12000003,Yorkshire and The Humber,7064.0,181496.1478
133559,2021-03-01,E12000003,Yorkshire and The Humber,9098.0,183307.8384
133560,2021-04-01,E12000003,Yorkshire and The Humber,6457.0,179601.8686
133561,2021-05-01,E12000003,Yorkshire and The Humber,5411.0,182339.4077


In [30]:

housing_sales_volume_2008=df_sales_volume[(df_sales_volume['Date'] > "2008-01-01") & (df_sales_volume['Date'] < '2010-01-01')]
housing_sales_volume_2008

,Date,AreaCode,RegionName,SalesVolume,AveragePrice
49,2008-02-01,S12000034,Aberdeenshire,368.0,170541.0575
50,2008-03-01,S12000034,Aberdeenshire,438.0,170689.2896
51,2008-04-01,S12000034,Aberdeenshire,425.0,169763.1431
52,2008-05-01,S12000034,Aberdeenshire,555.0,175577.9321
53,2008-06-01,S12000034,Aberdeenshire,409.0,176723.8804
...,...,...,...,...,...
133420,2009-08-01,E12000003,Yorkshire and The Humber,4767.0,132277.0186
133421,2009-09-01,E12000003,Yorkshire and The Humber,4868.0,132026.7427
133422,2009-10-01,E12000003,Yorkshire and The Humber,5601.0,134007.9386
133423,2009-11-01,E12000003,Yorkshire and The Humber,5106.0,134226.2793


In [37]:
#Merge the two data sets
merge_df_2020 = pd.merge(df_city_list,housing_sales_volume_2021_09, on="AreaCode")
merge_df_2020

,LAD,AreaCode,CTY,CTY_Code,Rgn,Rgn_Code,lat,long,Date,RegionName,SalesVolume,AveragePrice
0,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2020-02-01,Adur,72.0,306643.7957
1,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2020-03-01,Adur,92.0,310340.4646
2,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2020-04-01,Adur,35.0,318094.9621
3,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2020-05-01,Adur,34.0,319512.1572
4,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2020-06-01,Adur,57.0,312043.7535
...,...,...,...,...,...,...,...,...,...,...,...,...
6149,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2021-02-01,Wrexham,106.0,175100.7751
6150,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2021-03-01,Wrexham,176.0,176188.6691
6151,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2021-04-01,Wrexham,143.0,173402.2732
6152,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2021-05-01,Wrexham,117.0,174312.6521


In [38]:
merge_df_2020.to_csv('../Clean_csv/merge_df_2020.csv')

In [33]:
#Drop null values
merge_df = merge_df.dropna()
merge_df

,LAD,AreaCode,CTY,CTY_Code,Rgn,Rgn_Code,lat,long,Date,RegionName,SalesVolume,AveragePrice
0,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2020-02-01,Adur,72.0,306643.7957
1,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2020-03-01,Adur,92.0,310340.4646
2,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2020-04-01,Adur,35.0,318094.9621
3,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2020-05-01,Adur,34.0,319512.1572
4,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2020-06-01,Adur,57.0,312043.7535
...,...,...,...,...,...,...,...,...,...,...,...,...
6149,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2021-02-01,Wrexham,106.0,175100.7751
6150,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2021-03-01,Wrexham,176.0,176188.6691
6151,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2021-04-01,Wrexham,143.0,173402.2732
6152,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2021-05-01,Wrexham,117.0,174312.6521


In [39]:
#Merge the two data sets
merge_df_2008 = pd.merge(df_city_list,housing_sales_volume_2008, on="AreaCode")
merge_df_2008

,LAD,AreaCode,CTY,CTY_Code,Rgn,Rgn_Code,lat,long,Date,RegionName,SalesVolume,AveragePrice
0,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2008-02-01,Adur,66.0,220575.1156
1,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2008-03-01,Adur,53.0,220958.3928
2,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2008-04-01,Adur,91.0,217487.0336
3,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2008-05-01,Adur,95.0,216977.5992
4,Adur,E07000223,West Sussex,E10000032,South East,E12000008,50.854039,-0.307494,2008-06-01,Adur,66.0,215621.3152
...,...,...,...,...,...,...,...,...,...,...,...,...
8321,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2009-08-01,Wrexham,107.0,134061.6133
8322,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2009-09-01,Wrexham,107.0,133448.0507
8323,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2009-10-01,Wrexham,152.0,134645.9278
8324,Wrexham,W06000006,0,0,0,0,53.040668,-2.945333,2009-11-01,Wrexham,99.0,133164.4556


In [40]:
merge_df_2008.to_csv('../Clean_csv/merge_df_2008.csv')